In [2]:
import pandas as pd
from IPython.core.display import display, Markdown

# Convert raw data to a pandas data frame.
full = pd.DataFrame(
    {task.T: pd.Series(dict({'E': task.E, 'W': pbi.W(), 'S': task.S(), 'Timebox': str(task.Timebox())}))
     for pbi in backlog
     for task in pbi.tasks}).transpose()

full['Calendar'] = full.E.cumsum()

# Show a table with what you believe the weightiest item is (even if it's too large to do).
display(full.sort_values(by='W', ascending=False))

if any(pbi.E() > 1 for pbi in backlog):
    display(Markdown(
"""
Some of the highest priority tasks are larger than 1 hour. Before starting work, report your plan either:
- On a shared backlog
- At a daily stand-up
"""
))
    print("")

if any(pbi.E() > 7.5 for pbi in backlog):
    display(Markdown(
"""
Some of the highest priority tasks are larger than 8 hours. See:
- [Process: Backlog grooming](https://docs.google.com/document/d/1bmRN4n0kbMN2EOhMKk62VMjpkAYcolWbKj0vPLyJKa8/edit#)
- [Decision: Should you split story subtasks further?][1]
- [Process: PBI subtask planning](https://docs.google.com/document/d/1g39MM493y1KSkZLYHWtxmnf-68CZjfw_rPADFaErvgk/edit).

[1]: https://docs.google.com/document/d/1hQ99w3ZnrLpwygfZJHFTKfbrYqwo-cXB5tZIBmKtw4o/edit#
"""
))

from datetime import timedelta
if any((datetime.now() - pbi.creation_time) < timedelta(days=1, hours=0, seconds=1) for pbi in backlog):
    display(Markdown(
"""
Some of the highest priority tasks were created in the last day. See:
* [Process: Handling interruptions](https://docs.google.com/document/d/1Y0LbIWeP4wnwm09FsC2YejIMFvrG3wfNtiVQVQaG4ew/edit)
"""
))

# Plot (V, E) with a label on every point with the summary of the Task. You should be able to quickly see how many
# tasks or stories are small enough to start on (less than 4-8 hours). Hopefully you always have at least 3-4 stories
# that are small enough you can pick from. Over time you should be able to see what kind of V/E ratio you typically have
# on tasks you actually do.
nominal = pd.DataFrame({'W': full['W'].map(lambda w: w.nominal_value),
                        'E': full['E'].map(lambda e: e.nominal_value)})
   
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [8,8]

# https://stackoverflow.com/a/26000515/622049
# https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.errorbar.html
# https://matplotlib.org/gallery/lines_bars_and_markers/errorbar_limits_simple.html
# https://stackoverflow.com/a/43990689/622049
fig, ax = plt.subplots()
ax.errorbar(
    nominal.E,
    nominal.W,
    xerr=full['E'].map(lambda e: e.std_dev),
    yerr=full['W'].map(lambda w: w.std_dev),
    fmt='o')
ax.set_xlim(left=0.)
ax.set_ylim(bottom=0.)
ax.axvline(x=8, linestyle=':', color='red')
ax.axvline(x=1, linestyle='--', color='orange')

for k, v in nominal.iterrows():
    ax.annotate(s=k, xy=(v.E, v.W),
                xytext=(5,5), textcoords='offset points',
                family='sans-serif', fontsize=16, color='darkslategrey')

ax.grid()

NameError: name 'backlog' is not defined